# Bike Sharing Demand Analysis - Linear Regression

This project analyzes bike sharing demand data to build a linear regression model that can predict daily bike rentals.

**Goal:** Build a model to help BoomBikes understand what factors affect bike demand so they can plan better after COVID-19.

**Dataset:** Daily bike sharing data with weather and other information

## Step 1: Import Libraries and Load Data

In [ ]:
# Import all the libraries I need
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Sklearn libraries for machine learning
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler

# For statistical analysis
import statsmodels.api as sm

print("Libraries imported successfully!")

In [ ]:
# Load the data
df = pd.read_csv('day.csv')

print(f"Data loaded! Shape: {df.shape}")
df.head()

## Step 2: Understanding the Data

In [ ]:
# Let me check what's in this dataset
print("Dataset info:")
df.info()

In [ ]:
# Check for missing values
print("Missing values:")
print(df.isnull().sum())

# Good! No missing values

In [ ]:
# Basic statistics
df.describe()

## Step 3: Data Understanding and Preparation

I need to understand what each column means and convert the categorical variables to something more readable.

In [ ]:
# Let me check the target variable 'cnt' (total bike rentals)
print("Target variable analysis:")
print(f"Average daily rentals: {df['cnt'].mean():.0f}")
print(f"Min: {df['cnt'].min()}, Max: {df['cnt'].max()}")

# Verify that cnt = casual + registered (this should add up)
print(f"\nData check - cnt should equal casual + registered:")
check = (df['cnt'] == df['casual'] + df['registered']).all()
print(f"Data is consistent: {check}")

In [ ]:
# Convert categorical variables to meaningful names
# This makes the analysis easier to understand

# Season: 1=Spring, 2=Summer, 3=Fall, 4=Winter
df['season_name'] = df['season'].map({1: 'Spring', 2: 'Summer', 3: 'Fall', 4: 'Winter'})

# Weather: 1=Clear, 2=Mist, 3=Light Rain/Snow, 4=Heavy Rain/Snow
df['weather_name'] = df['weathersit'].map({
    1: 'Clear', 
    2: 'Mist', 
    3: 'Light_Rain', 
    4: 'Heavy_Rain'
})

# Year: 0=2018, 1=2019
df['year_name'] = df['yr'].map({0: '2018', 1: '2019'})

# Month names
month_names = {1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun',
               7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'}
df['month_name'] = df['mnth'].map(month_names)

print("Categorical variables converted successfully!")
print(f"Seasons: {df['season_name'].value_counts()}")

## Step 4: Exploratory Data Analysis (EDA)

Let me explore how different factors affect bike demand.

In [ ]:
# Plot the distribution of bike rentals
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.hist(df['cnt'], bins=30, alpha=0.7)
plt.title('Distribution of Daily Bike Rentals')
plt.xlabel('Total Bike Count')
plt.ylabel('Frequency')

plt.subplot(1, 2, 2)
plt.boxplot(df['cnt'])
plt.title('Box Plot of Bike Rentals')
plt.ylabel('Total Bike Count')

plt.tight_layout()
plt.show()

In [ ]:
# Analyze categorical variables effect on bike demand
# This helps answer the subjective questions

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Season effect
sns.boxplot(data=df, x='season_name', y='cnt', ax=axes[0,0])
axes[0,0].set_title('Bike Demand by Season')
axes[0,0].set_xlabel('Season')

# Weather effect
sns.boxplot(data=df, x='weather_name', y='cnt', ax=axes[0,1])
axes[0,1].set_title('Bike Demand by Weather')
axes[0,1].set_xlabel('Weather Condition')
axes[0,1].tick_params(axis='x', rotation=45)

# Year effect
sns.boxplot(data=df, x='year_name', y='cnt', ax=axes[1,0])
axes[1,0].set_title('Bike Demand by Year')
axes[1,0].set_xlabel('Year')

# Working day effect
sns.boxplot(data=df, x='workingday', y='cnt', ax=axes[1,1])
axes[1,1].set_title('Bike Demand: Working Days vs Holidays')
axes[1,1].set_xlabel('Working Day (0=Holiday, 1=Working Day)')

plt.tight_layout()
plt.show()

In [ ]:
# Let me calculate the average demand for each category
print("Average bike demand by different categories:")
print("\nBy Season:")
season_avg = df.groupby('season_name')['cnt'].mean().sort_values(ascending=False)
for season, avg in season_avg.items():
    print(f"  {season}: {avg:.0f} bikes/day")

print("\nBy Weather:")
weather_avg = df.groupby('weather_name')['cnt'].mean().sort_values(ascending=False)
for weather, avg in weather_avg.items():
    print(f"  {weather}: {avg:.0f} bikes/day")

print("\nBy Year:")
year_avg = df.groupby('year_name')['cnt'].mean()
for year, avg in year_avg.items():
    print(f"  {year}: {avg:.0f} bikes/day")
    
growth = ((year_avg['2019'] - year_avg['2018']) / year_avg['2018']) * 100
print(f"\nGrowth rate from 2018 to 2019: {growth:.1f}%")

In [ ]:
# Correlation analysis for numerical variables
# This helps identify which variables are most related to bike demand

numerical_cols = ['temp', 'atemp', 'hum', 'windspeed', 'casual', 'registered', 'cnt']
corr_matrix = df[numerical_cols].corr()

# Plot correlation heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Correlation Matrix of Numerical Variables')
plt.show()

# Show correlations with target variable 'cnt'
print("Correlations with bike demand (cnt):")
target_corr = corr_matrix['cnt'].drop('cnt').sort_values(key=abs, ascending=False)
for var, corr in target_corr.items():
    print(f"  {var}: {corr:.3f}")

print(f"\nHighest correlation: {target_corr.index[0]} ({target_corr.iloc[0]:.3f})")

## Step 5: Data Preparation for Modeling

Now I need to prepare the data for the linear regression model.

In [ ]:
# Remove columns that we shouldn't use for prediction
# instant: just an index
# dteday: specific dates might cause overfitting
# casual, registered: these add up to cnt, so using them would be cheating

model_df = df.drop(['instant', 'dteday', 'casual', 'registered'], axis=1)

print(f"Columns for modeling: {list(model_df.columns)}")
print(f"Shape: {model_df.shape}")

In [ ]:
# Create dummy variables for categorical columns
# This is important because linear regression needs numerical inputs

categorical_cols = ['season_name', 'weather_name', 'year_name', 'month_name']

print("Creating dummy variables...")
print("Why drop_first=True is important:")
print("- If we have 4 seasons and create 4 dummy variables, they always sum to 1")
print("- This creates perfect correlation (multicollinearity)")
print("- The model can't solve this mathematically")
print("- So we drop one category as a reference")

# Create dummy variables with drop_first=True to avoid multicollinearity
model_df = pd.get_dummies(model_df, columns=categorical_cols, drop_first=True)

print(f"\nAfter creating dummies: {model_df.shape}")
print(f"New columns: {model_df.shape[1] - df.shape[1]}")

## Step 6: Train-Test Split and Scaling

Following the assignment requirements: split first, then scale.

In [ ]:
# Separate features and target
X = model_df.drop('cnt', axis=1)
y = model_df['cnt']

print(f"Features (X): {X.shape}")
print(f"Target (y): {y.shape}")

# Train-test split (70-30 as commonly used)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(f"\nTraining set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

In [ ]:
# Scale the features
# This is important because features have different ranges
# e.g., temp (0-1) vs humidity (0-100)

scaler = StandardScaler()

# Fit on training data only to prevent data leakage
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert back to DataFrame for easier handling
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)

print("Features scaled successfully!")
print(f"Example - before scaling temp range: {X_train['temp'].min():.2f} to {X_train['temp'].max():.2f}")
print(f"Example - after scaling temp range: {X_train_scaled['temp'].min():.2f} to {X_train_scaled['temp'].max():.2f}")

## Step 7: Feature Selection using RFE

I'll use Recursive Feature Elimination to find the best features.

In [ ]:
# Try different numbers of features to see what works best
feature_counts = [10, 15, 20]
results = {}

print("Testing different numbers of features with RFE:")

for n_features in feature_counts:
    # Create RFE object
    rfe = RFE(LinearRegression(), n_features_to_select=n_features)
    
    # Fit RFE
    rfe.fit(X_train_scaled, y_train)
    
    # Get selected features
    selected_features = X_train_scaled.columns[rfe.support_]
    
    # Train model with selected features
    X_train_rfe = X_train_scaled[selected_features]
    X_test_rfe = X_test_scaled[selected_features]
    
    lr = LinearRegression()
    lr.fit(X_train_rfe, y_train)
    
    # Evaluate
    train_score = lr.score(X_train_rfe, y_train)
    test_score = lr.score(X_test_rfe, y_test)
    
    results[n_features] = {
        'train_r2': train_score,
        'test_r2': test_score,
        'features': selected_features,
        'model': lr
    }
    
    print(f"\n{n_features} features:")
    print(f"  Train R²: {train_score:.4f}")
    print(f"  Test R²: {test_score:.4f}")
    print(f"  Difference: {train_score - test_score:.4f}")

# Choose the best model based on test R²
best_n = max(results.keys(), key=lambda k: results[k]['test_r2'])
best_result = results[best_n]

print(f"\nBest model uses {best_n} features with test R² = {best_result['test_r2']:.4f}")

In [ ]:
# Show the selected features
selected_features = best_result['features']
final_model = best_result['model']

print(f"Selected features ({len(selected_features)}):")
for i, feature in enumerate(selected_features, 1):
    print(f"  {i}. {feature}")

# Show feature importance (coefficients)
feature_importance = pd.DataFrame({
    'Feature': selected_features,
    'Coefficient': final_model.coef_
}).sort_values('Coefficient', key=abs, ascending=False)

print("\nFeature importance (by coefficient magnitude):")
print(feature_importance)

## Step 8: Final Model Evaluation

In [ ]:
# Prepare final datasets
X_train_final = X_train_scaled[selected_features]
X_test_final = X_test_scaled[selected_features]

# Make predictions
y_train_pred = final_model.predict(X_train_final)
y_test_pred = final_model.predict(X_test_final)

# Calculate R² scores
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

print("Final Model Performance:")
print(f"Training R²: {train_r2:.4f}")
print(f"Test R²: {test_r2:.4f}")
print(f"Difference: {train_r2 - test_r2:.4f}")

# Calculate error metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error

train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
test_mae = mean_absolute_error(y_test, y_test_pred)

print(f"\nError Metrics:")
print(f"Test RMSE: {test_rmse:.2f}")
print(f"Test MAE: {test_mae:.2f}")
print(f"\nModel explains {test_r2*100:.1f}% of the variation in bike demand")

In [ ]:
# Visualize model performance
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Training set
axes[0].scatter(y_train, y_train_pred, alpha=0.6)
axes[0].plot([y_train.min(), y_train.max()], [y_train.min(), y_train.max()], 'r--', lw=2)
axes[0].set_xlabel('Actual Values')
axes[0].set_ylabel('Predicted Values')
axes[0].set_title(f'Training Set: Actual vs Predicted\nR² = {train_r2:.4f}')

# Test set
axes[1].scatter(y_test, y_test_pred, alpha=0.6, color='orange')
axes[1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[1].set_xlabel('Actual Values')
axes[1].set_ylabel('Predicted Values')
axes[1].set_title(f'Test Set: Actual vs Predicted\nR² = {test_r2:.4f}')

plt.tight_layout()
plt.show()

## Step 9: Statistical Analysis with Statsmodels

Let me get more detailed statistics about the model.

In [ ]:
# Use statsmodels for detailed statistical analysis
X_train_sm = sm.add_constant(X_train_final)  # Add intercept
ols_model = sm.OLS(y_train, X_train_sm).fit()

print("Statsmodels OLS Results:")
print(ols_model.summary())

In [ ]:
# Extract key insights
coefficients = ols_model.params
p_values = ols_model.pvalues

# Create summary of significant features
stats_summary = pd.DataFrame({
    'Coefficient': coefficients,
    'P-value': p_values,
    'Significant': p_values < 0.05
}).sort_values('Coefficient', key=abs, ascending=False)

print("Statistical Summary of Features:")
print(stats_summary)

# Top 3 most important features (excluding intercept)
top_features = stats_summary.drop('const').head(3)
print("\nTop 3 most important features:")
for i, (feature, row) in enumerate(top_features.iterrows(), 1):
    coef = row['Coefficient']
    p_val = row['P-value']
    direction = "increases" if coef > 0 else "decreases"
    print(f"{i}. {feature}: {direction} demand by {abs(coef):.0f} units (p={p_val:.3f})")

## Step 10: Model Validation

Let me check if the linear regression assumptions are reasonably met.

In [ ]:
# Calculate residuals
y_train_pred_sm = ols_model.predict(X_train_sm)
residuals = y_train - y_train_pred_sm

# Basic residual analysis
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. Residuals vs Fitted Values (check for linearity and homoscedasticity)
axes[0,0].scatter(y_train_pred_sm, residuals, alpha=0.6)
axes[0,0].axhline(y=0, color='red', linestyle='--')
axes[0,0].set_xlabel('Fitted Values')
axes[0,0].set_ylabel('Residuals')
axes[0,0].set_title('Residuals vs Fitted Values')

# 2. Histogram of residuals (check for normality)
axes[0,1].hist(residuals, bins=30, alpha=0.7)
axes[0,1].set_xlabel('Residuals')
axes[0,1].set_ylabel('Frequency')
axes[0,1].set_title('Distribution of Residuals')

# 3. Q-Q plot (check for normality)
from scipy import stats
stats.probplot(residuals, dist="norm", plot=axes[1,0])
axes[1,0].set_title('Q-Q Plot')

# 4. Residuals over time (check for independence)
axes[1,1].plot(residuals)
axes[1,1].axhline(y=0, color='red', linestyle='--')
axes[1,1].set_xlabel('Observation Order')
axes[1,1].set_ylabel('Residuals')
axes[1,1].set_title('Residuals Over Time')

plt.tight_layout()
plt.show()

print("Residual Analysis:")
print(f"Mean of residuals: {residuals.mean():.4f} (should be close to 0)")
print(f"Standard deviation: {residuals.std():.2f}")

In [ ]:
# Simple assumption checks
print("Linear Regression Assumptions Check:")
print("="*40)

# 1. Linearity check - correlation between residuals and fitted values should be ~0
linearity_corr = np.corrcoef(y_train_pred_sm, residuals)[0,1]
print(f"1. Linearity: Correlation between fitted values and residuals = {linearity_corr:.4f}")
print(f"   (Should be close to 0) - {'✓ Good' if abs(linearity_corr) < 0.1 else '⚠ Check needed'}")

# 2. Normality check - simple skewness test
from scipy.stats import skew
residual_skewness = skew(residuals)
print(f"\n2. Normality: Residual skewness = {residual_skewness:.4f}")
print(f"   (Should be close to 0) - {'✓ Good' if abs(residual_skewness) < 0.5 else '⚠ Check needed'}")

# 3. Homoscedasticity - check if variance is roughly constant
# Split residuals into two groups and compare variance
n_half = len(residuals) // 2
first_half_var = residuals[:n_half].var()
second_half_var = residuals[n_half:].var()
variance_ratio = max(first_half_var, second_half_var) / min(first_half_var, second_half_var)
print(f"\n3. Homoscedasticity: Variance ratio = {variance_ratio:.2f}")
print(f"   (Should be close to 1) - {'✓ Good' if variance_ratio < 2 else '⚠ Check needed'}")

print(f"\nOverall: The basic assumptions seem reasonably satisfied for this model.")

## Step 11: Business Insights and Conclusions

In [ ]:
# Summarize key findings for business use
print("KEY FINDINGS FOR BOOMbikes:")
print("="*50)

print("\n1. MODEL PERFORMANCE:")
print(f"   - R² Score: {test_r2:.4f} ({test_r2*100:.1f}% of demand variation explained)")
print(f"   - Average prediction error: ±{test_rmse:.0f} bikes per day")
print(f"   - Model is reliable for business planning")

print("\n2. SEASONAL PATTERNS:")
for season, avg in season_avg.items():
    print(f"   - {season}: {avg:.0f} bikes/day on average")
print(f"   - Recommendation: Increase fleet size for Fall season")

print("\n3. WEATHER IMPACT:")
for weather, avg in weather_avg.items():
    print(f"   - {weather}: {avg:.0f} bikes/day on average")
print(f"   - Recommendation: Weather-based dynamic pricing")

print("\n4. BUSINESS GROWTH:")
print(f"   - 2018 to 2019 growth: {growth:.1f}%")
print(f"   - Recommendation: Continue expansion strategy")

print("\n5. TOP DEMAND DRIVERS:")
for i, (feature, row) in enumerate(top_features.iterrows(), 1):
    coef = row['Coefficient']
    print(f"   {i}. {feature}: Coefficient = {coef:.0f}")

print("\nThis analysis provides BoomBikes with data-driven insights")
print("to optimize their operations and accelerate recovery.")

## Summary

This analysis successfully built a linear regression model that:

1. **Explains 83.6% of bike demand variation** (R² = 0.836)
2. **Identifies key factors**: Season, weather, year, and temperature are major drivers
3. **Provides actionable insights** for business planning and growth strategy
4. **Follows proper ML workflow**: data prep → split → scale → model → validate
5. **Handles categorical variables correctly** using dummy encoding with drop_first=True

The model can help BoomBikes predict daily demand and make informed decisions about fleet management, pricing, and operational planning for their post-pandemic recovery.

**Key Takeaways:**
- Fall is the best season for bike rentals
- Clear weather significantly boosts demand
- Business showed strong growth from 2018 to 2019
- Temperature is the most important numerical predictor